# Import

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

from torchvision.transforms import Compose, ToPILImage, Resize, ToTensor, Normalize
import torchvision.transforms as transforms
# from torchvision.transforms import RandomResizedCrop, RandomHorizontalFlip, RandomVerticalFlip, RandomAffine, RandomErasing, GaussianBlur

import timm

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

from tqdm import tqdm

from PIL import Image, ImageEnhance, ImageOps
import random

# Config / Hyperparams

In [2]:
MODEL_NAME = 'resnet50.ra_in1k'
# TRIAL_NAME = f'250314_{MODEL_NAME}_5fold'
TRIAL_NAME = f'250421_{MODEL_NAME}_5fold_10xEpoch_seed1'
TRAINING = True

N_EPOCHS = 10000
BATCH_SIZE = 80
LR = 3e-4
N_FOLDS = 5
SEED = 1
STOP_THRESHOLD = 500
IN_MEMORY_MODELS = False
BEST_ACC = False
ES_COOLDOWN = 200 # N_EPOCHS//5
CLASSES = 10
ENSEMBLE='sv'
TTA = True
VAL_TTA = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
def set_seed(seed: int):
    random.seed(seed)  
    np.random.seed(seed) 
    torch.manual_seed(seed) 
    torch.cuda.manual_seed(seed) 
    torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = False  
set_seed(SEED)

# Cifar-10 Auto Augmentation values and functions

In [4]:
class ShearX(object):
    def __init__(self, fillcolor=(128)):
        self.fillcolor = fillcolor

    def __call__(self, x, magnitude):
        return x.transform(
            x.size, Image.AFFINE, (1, magnitude * random.choice([-1, 1]), 0, 0, 1, 0),
            Image.BICUBIC, fillcolor=self.fillcolor)


class ShearY(object):
    def __init__(self, fillcolor=(128)):
        self.fillcolor = fillcolor

    def __call__(self, x, magnitude):
        return x.transform(
            x.size, Image.AFFINE, (1, 0, 0, magnitude * random.choice([-1, 1]), 1, 0),
            Image.BICUBIC, fillcolor=self.fillcolor)


class TranslateX(object):
    def __init__(self, fillcolor=(128)):
        self.fillcolor = fillcolor

    def __call__(self, x, magnitude):
        return x.transform(
            x.size, Image.AFFINE, (1, 0, magnitude * x.size[0] * random.choice([-1, 1]), 0, 1, 0),
            fillcolor=self.fillcolor)


class TranslateY(object):
    def __init__(self, fillcolor=(128)):
        self.fillcolor = fillcolor

    def __call__(self, x, magnitude):
        return x.transform(
            x.size, Image.AFFINE, (1, 0, 0, 0, 1, magnitude * x.size[1] * random.choice([-1, 1])),
            fillcolor=self.fillcolor)


class Rotate(object):
    # from https://stackoverflow.com/questions/
    # 5252170/specify-image-filling-color-when-rotating-in-python-with-pil-and-setting-expand
    def __call__(self, x, magnitude):
        rot = x.convert("RGBA").rotate(magnitude * random.choice([-1, 1]))
        return Image.composite(rot, Image.new("RGBA", rot.size, (128,) * 4), rot).convert(x.mode)


class Color(object):
    def __call__(self, x, magnitude):
        return ImageEnhance.Color(x).enhance(1 + magnitude * random.choice([-1, 1]))


class Posterize(object):
    def __call__(self, x, magnitude):
        return ImageOps.posterize(x, magnitude)


class Solarize(object):
    def __call__(self, x, magnitude):
        return ImageOps.solarize(x, magnitude)


class Contrast(object):
    def __call__(self, x, magnitude):
        return ImageEnhance.Contrast(x).enhance(1 + magnitude * random.choice([-1, 1]))


class Sharpness(object):
    def __call__(self, x, magnitude):
        return ImageEnhance.Sharpness(x).enhance(1 + magnitude * random.choice([-1, 1]))


class Brightness(object):
    def __call__(self, x, magnitude):
        return ImageEnhance.Brightness(x).enhance(1 + magnitude * random.choice([-1, 1]))


class AutoContrast(object):
    def __call__(self, x, magnitude):
        return ImageOps.autocontrast(x)


class Equalize(object):
    def __call__(self, x, magnitude):
        return ImageOps.equalize(x)


class Invert(object):
    def __call__(self, x, magnitude):
        return ImageOps.invert(x)

class SubPolicy(object):
    def __init__(self, p1, operation1, magnitude_idx1, p2, operation2, magnitude_idx2, fillcolor=(128)):
        ranges = {
            "shearX": np.linspace(0, 0.3, 10),
            "shearY": np.linspace(0, 0.3, 10),
            "translateX": np.linspace(0, 150 / 331, 10),
            "translateY": np.linspace(0, 150 / 331, 10),
            "rotate": np.linspace(0, 30, 10),
            "color": np.linspace(0.0, 0.9, 10),
            "posterize": np.round(np.linspace(8, 4, 10), 0).astype(np.int_),
            "solarize": np.linspace(256, 0, 10),
            "contrast": np.linspace(0.0, 0.9, 10),
            "sharpness": np.linspace(0.0, 0.9, 10),
            "brightness": np.linspace(0.0, 0.9, 10),
            "autocontrast": [0] * 10,
            "equalize": [0] * 10,
            "invert": [0] * 10
        }

        func = {
            "shearX": ShearX(fillcolor=fillcolor),
            "shearY": ShearY(fillcolor=fillcolor),
            "translateX": TranslateX(fillcolor=fillcolor),
            "translateY": TranslateY(fillcolor=fillcolor),
            "rotate": Rotate(),
            "color": Color(),
            "posterize": Posterize(),
            "solarize": Solarize(),
            "contrast": Contrast(),
            "sharpness": Sharpness(),
            "brightness": Brightness(),
            "autocontrast": AutoContrast(),
            "equalize": Equalize(),
            "invert": Invert()
        }

        self.p1 = p1
        self.operation1 = func[operation1]
        self.magnitude1 = ranges[operation1][magnitude_idx1]
        self.p2 = p2
        self.operation2 = func[operation2]
        self.magnitude2 = ranges[operation2][magnitude_idx2]

    def __call__(self, img):
        if random.random() < self.p1:
            img = self.operation1(img, self.magnitude1)
        if random.random() < self.p2:
            img = self.operation2(img, self.magnitude2)
        return img

class CIFAR10Policy(object):
    """ Randomly choose one of the best 25 Sub-policies on CIFAR10.

        Example:
        >>> policy = CIFAR10Policy()
        >>> transformed = policy(image)

        Example as a PyTorch Transform:
        >>> transform=transforms.Compose([
        >>>     transforms.Resize(256),
        >>>     CIFAR10Policy(),
        >>>     transforms.ToTensor()])
    """
    def __init__(self, fillcolor=(128)):
        self.policies = [
            SubPolicy(0.1, "invert", 7, 0.2, "contrast", 6, fillcolor),
            SubPolicy(0.7, "rotate", 2, 0.3, "translateX", 9, fillcolor),
            SubPolicy(0.8, "sharpness", 1, 0.9, "sharpness", 3, fillcolor),
            SubPolicy(0.5, "shearY", 8, 0.7, "translateY", 9, fillcolor),
            SubPolicy(0.5, "autocontrast", 8, 0.9, "equalize", 2, fillcolor),

            SubPolicy(0.2, "shearY", 7, 0.3, "posterize", 7, fillcolor),
            SubPolicy(0.4, "color", 3, 0.6, "brightness", 7, fillcolor),
            SubPolicy(0.3, "sharpness", 9, 0.7, "brightness", 9, fillcolor),
            SubPolicy(0.6, "equalize", 5, 0.5, "equalize", 1, fillcolor),
            SubPolicy(0.6, "contrast", 7, 0.6, "sharpness", 5, fillcolor),

            SubPolicy(0.7, "color", 7, 0.5, "translateX", 8, fillcolor),
            SubPolicy(0.3, "equalize", 7, 0.4, "autocontrast", 8, fillcolor),
            SubPolicy(0.4, "translateY", 3, 0.2, "sharpness", 6, fillcolor),
            SubPolicy(0.9, "brightness", 6, 0.2, "color", 8, fillcolor),
            SubPolicy(0.5, "solarize", 2, 0.0, "invert", 3, fillcolor),

            SubPolicy(0.2, "equalize", 0, 0.6, "autocontrast", 0, fillcolor),
            SubPolicy(0.2, "equalize", 8, 0.6, "equalize", 4, fillcolor),
            SubPolicy(0.9, "color", 9, 0.6, "equalize", 6, fillcolor),
            SubPolicy(0.8, "autocontrast", 4, 0.2, "solarize", 8, fillcolor),
            SubPolicy(0.1, "brightness", 3, 0.7, "color", 0, fillcolor),

            SubPolicy(0.4, "solarize", 5, 0.9, "autocontrast", 3, fillcolor),
            SubPolicy(0.9, "translateY", 9, 0.7, "translateY", 9, fillcolor),
            SubPolicy(0.9, "autocontrast", 2, 0.8, "solarize", 3, fillcolor),
            SubPolicy(0.8, "equalize", 8, 0.1, "invert", 3, fillcolor),
            SubPolicy(0.7, "translateY", 9, 0.9, "autocontrast", 1, fillcolor)
        ]

    def __call__(self, img):
        policy_idx = random.randint(0, len(self.policies) - 1)
        return self.policies[policy_idx](img)

    def __repr__(self):
        return "AutoAugment CIFAR10 Policy"

In [5]:
class Cutout(object):
    """Randomly mask out one or more patches from an image.

    Args:
        n_holes (int): Number of patches to cut out of each image.
        length (int): The length (in pixels) of each square patch.
    """
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        """
        Args:
            img (Tensor): Tensor image of size (C, H, W).
        Returns:
            Tensor: Image with n_holes of dimension length x length cut out of it.
        """
        h = img.size(1)
        w = img.size(2)

        mask = np.ones((h, w), np.float32)

        for n in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)

            mask[y1: y2, x1: x2] = 0.

        mask = torch.from_numpy(mask)
        mask = mask.expand_as(img)
        img = img * mask

        return img

# Data Loading

In [6]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

encoder = LabelEncoder()
train['label'] = encoder.fit_transform(train['label'])

skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

# Datasets & DataLoaders & criterion setup

In [7]:
class CustomDataset(Dataset):
    def __init__(self, pixel_df, label_df=None, transform=None):
        self.pixel_df = pixel_df.reset_index(drop=True)
        self.label_df = label_df.reset_index(drop=True) if label_df is not None else None
        self.transform = transform

    def __len__(self):
        return len(self.pixel_df)
    
    def __getitem__(self, idx):
        # Reshape to (32, 32) from flattened data
        image = self.pixel_df.iloc[idx].values.astype(np.uint8).reshape(32, 32)
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # shape: (1, 32, 32)

        if self.transform:
            image = self.transform(image)

        if self.label_df is not None:
            label = torch.tensor(self.label_df.iloc[idx], dtype=torch.long)
            return image, label
        else:
            return image
        
# RandomResizedCrop((224,224), scale=(0.8, 1)),
# transforms.RandomAffine(
#     degrees=(-180, 180),
#     translate=(0.4, 0.4),
#     scale=(0.8, 1.2),
#     shear=(-20, 20)
# ),

train_transform = Compose([
    ToPILImage(),
    transforms.RandomCrop(32, padding=4),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    CIFAR10Policy(),
    ToTensor(),
    Cutout(n_holes=1, length=16),
    Normalize(mean = [0.5], std=[0.5]),
])

test_transform = Compose([
    ToPILImage(),
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean = [0.5], std=[0.5]),
])

if TTA:
    tta_transforms = [
        transforms.Compose([transforms.RandomHorizontalFlip(p=1.0)]),
        transforms.Compose([transforms.RandomVerticalFlip(p=1.0)]),
        transforms.Compose([transforms.RandomRotation((0, 30))]),
    ]

loader_params = {
    'batch_size': BATCH_SIZE,
    'num_workers': 8,
    'pin_memory': True
}

test_loader_params = {
    'batch_size': 1,
    'num_workers': 8,
    'pin_memory': True
}

criterion = nn.CrossEntropyLoss()


# Training / Validation Functions

In [8]:
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(loader.dataset)
    return epoch_loss

def validate_one_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct, total = 0, 0
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)

            if VAL_TTA:
                tta_images = [tta_transform(images) for tta_transform in tta_transforms]
                tta_outputs = [model(tta_image) for tta_image in tta_images]
                outputs = [outputs] + tta_outputs
                outputs = torch.stack(outputs).mean(dim=0)

            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(loader.dataset)
    accuracy = correct / total
    return epoch_loss, accuracy

def hv_validate_one_epoch(models, loader, criterion, device):
    for model in models:
        model.eval()
    running_loss = 0.0
    correct, total = 0, 0
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            
            sf_outputs = torch.zeros((images.size(0), CLASSES)).to(device)
            outputs = torch.zeros((images.size(0), CLASSES)).to(device)
            for model in models:
                output = model(images)

                if TTA:
                    tta_images = [tta_transform(images) for tta_transform in tta_transforms]
                    tta_outputs = [model(tta_image) for tta_image in tta_images]
                    output = [output] + tta_outputs
                    output = torch.stack(output).mean(dim=0)

                # soft-voting
                sf_outputs = sf_outputs + output

                # hard-voting
                _, predict_indices = torch.max(output.data, dim=1)
                pred_one_hot = torch.zeros((images.shape[0], CLASSES)).to(device)
                pred_one_hot.scatter_(1, predict_indices.unsqueeze(1), 1)
                outputs = outputs + pred_one_hot

            max_val, _ = torch.max(outputs, dim=1, keepdim=True)
            cnt = torch.sum(outputs == max_val, dim=1)
            for i in range(cnt.shape[0]):
                if cnt[i] > 1:
                    output[i] = sf_outputs[i]
            outputs = outputs / len(models)
            
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(loader.dataset)
    accuracy = correct / total
    return epoch_loss, accuracy

def sv_validate_one_epoch(models, loader, criterion, device):
    for model in models:
        model.eval()
    running_loss = 0.0
    correct, total = 0, 0
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = torch.zeros((images.size(0), CLASSES)).to(device)
            for model in models:
                output = model(images)

                if TTA:
                    tta_images = [tta_transform(images) for tta_transform in tta_transforms]
                    tta_outputs = [model(tta_image) for tta_image in tta_images]
                    output = [output] + tta_outputs
                    output = torch.stack(output).mean(dim=0)

                outputs = outputs + output
            outputs = outputs / len(models)
            
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            
            _, predicted = torch.max(outputs, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(loader.dataset)
    accuracy = correct / total
    return epoch_loss, accuracy


# K-fold training

In [9]:
if TRAINING:
    models = []

    for i, (train_idx, valid_idx) in enumerate(skf.split(train.iloc[:, 2:], train['label'])):
        train_dataset = CustomDataset(pixel_df=train.iloc[train_idx, 2:], label_df=train.iloc[train_idx, 1], transform=train_transform)
        valid_dataset = CustomDataset(pixel_df=train.iloc[valid_idx, 2:], label_df=train.iloc[valid_idx, 1], transform=test_transform)

        train_loader = DataLoader(train_dataset, shuffle=True, **loader_params)
        valid_loader = DataLoader(valid_dataset, shuffle=False, **loader_params)

        model = timm.create_model(
            model_name=MODEL_NAME,
            pretrained=False,
            num_classes=CLASSES,
            in_chans=1
        ).to(device)

        optimizer = optim.AdamW(model.parameters(), lr=LR)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=N_EPOCHS)

        best_loss = float('inf')
        best_acc = 0.0
        best_model = None
        
        stop_count = 0

        for epoch in range(N_EPOCHS):
            train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
            val_loss, val_acc = validate_one_epoch(model, valid_loader, criterion, device)
            
            if BEST_ACC:
                print(f"\rEpoch [{epoch+1}/{N_EPOCHS}] Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Accuracy: {val_acc*100:.4f}% | Best Accuracy: {best_acc*100:.4f}% | Best loss: {best_loss:.4f} | {stop_count}/{STOP_THRESHOLD} ES", end='')
                if val_acc > best_acc:
                    best_acc = val_acc
                    best_loss = val_loss
                    best_model = model
                    stop_count = 0
                    torch.save(best_model.state_dict(), f'./{TRIAL_NAME}_{i}_model_weight.pth')
            else:
                print(f"\rEpoch [{epoch+1}/{N_EPOCHS}] Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Accuracy: {val_acc*100:.4f}% | Best Accuracy: {best_acc*100:.4f}% | Best loss: {best_loss:.4f} | {stop_count}/{STOP_THRESHOLD} ES", end='')
                if val_loss < best_loss:
                    best_acc = val_acc
                    best_loss = val_loss
                    best_model = model
                    stop_count = 0
                    torch.save(best_model.state_dict(), f'./{TRIAL_NAME}_{i}_model_weight.pth')
        
            if epoch > ES_COOLDOWN and stop_count >= STOP_THRESHOLD:
                break

            if type(scheduler) == optim.lr_scheduler.ReduceLROnPlateau:
                scheduler.step(val_loss)
            else:
                scheduler.step()

            if epoch > ES_COOLDOWN:
                stop_count += 1

        if IN_MEMORY_MODELS:
            models.append(best_model)
        
        print(f'\n{i}th-fold model Acc: {best_acc*100:.4f}%, Loss: {best_loss}')
        

Epoch [1218/10000] Train Loss: 0.0491 | Val Loss: 0.2264 | Val Accuracy: 95.4545% | Best Accuracy: 99.3506% | Best loss: 0.0227 | 500/500 ES
0th-fold model Acc: 99.3506%, Loss: 0.022699884767946484
Epoch [1888/10000] Train Loss: 0.0226 | Val Loss: 0.1364 | Val Accuracy: 98.0519% | Best Accuracy: 100.0000% | Best loss: 0.0058 | 500/500 ESS
1th-fold model Acc: 100.0000%, Loss: 0.005849817035985845
Epoch [2456/10000] Train Loss: 0.0346 | Val Loss: 0.0620 | Val Accuracy: 98.7013% | Best Accuracy: 99.3506% | Best loss: 0.0118 | 500/500 ES
2th-fold model Acc: 99.3506%, Loss: 0.011825455872753224
Epoch [2903/10000] Train Loss: 0.0446 | Val Loss: 0.3717 | Val Accuracy: 95.4545% | Best Accuracy: 98.0519% | Best loss: 0.1062 | 500/500 ES
3th-fold model Acc: 98.0519%, Loss: 0.10618829039377523
Epoch [1184/10000] Train Loss: 0.0591 | Val Loss: 0.1997 | Val Accuracy: 98.0392% | Best Accuracy: 98.0392% | Best loss: 0.0487 | 500/500 ES
4th-fold model Acc: 98.0392%, Loss: 0.0486677436316325


# Bagging Inference

In [10]:
if ENSEMBLE == 'sv':
    models = []
    for i in range(N_FOLDS):
        model_state_dict = torch.load(f'./{TRIAL_NAME}_{i}_model_weight.pth',)
        model = timm.create_model(
            model_name=MODEL_NAME,
            pretrained=False,
            num_classes=CLASSES,
            in_chans=1
        ).to(device)
        model.load_state_dict(model_state_dict)
        models.append(model)

    total_dataset = CustomDataset(pixel_df=train.iloc[: , 2:], label_df=train.iloc[: , 1], transform=test_transform)
    total_loader = DataLoader(total_dataset, shuffle=True, **loader_params)

    total_loss, total_acc = sv_validate_one_epoch(models, total_loader, criterion, device)
    print(f"SV model's Test Loss: {total_loss:.4f} | Test Accuracy: {total_acc*100:.4f}%")

    test_dataset = CustomDataset(pixel_df=test.iloc[:, 1:], transform=test_transform)
    test_loader = DataLoader(test_dataset, shuffle=False, **test_loader_params)

    preds = []
    with torch.no_grad():
        for images in test_loader:
            images = images.to(device)
            outputs = torch.zeros((images.size(0), CLASSES)).to(device)
            for model in models:
                output = model(images)

                if TTA:
                    tta_images = [tta_transform(images) for tta_transform in tta_transforms]
                    tta_outputs = [model(tta_image) for tta_image in tta_images]
                    output = [output] + tta_outputs
                    output = torch.stack(output).mean(dim=0)

                outputs = outputs + output
            outputs = outputs / len(models)
            _, predicted = torch.max(outputs.data, 1)
            preds.extend(predicted.cpu().numpy())

    # Decode predictions
    pred_labels = encoder.inverse_transform(preds)

if ENSEMBLE == 'hv':
    models = []
    for i in range(N_FOLDS):
        model_state_dict = torch.load(f'./{TRIAL_NAME}_{i}_model_weight.pth',)
        model = timm.create_model(
            model_name=MODEL_NAME,
            pretrained=False,
            num_classes=CLASSES,
            in_chans=1
        ).to(device)
        model.load_state_dict(model_state_dict)
        models.append(model)

    total_dataset = CustomDataset(pixel_df=train.iloc[: , 2:], label_df=train.iloc[: , 1], transform=test_transform)
    total_loader = DataLoader(total_dataset, shuffle=True, **loader_params)

    total_loss, total_acc = hv_validate_one_epoch(models, total_loader, criterion, device)
    print(f"HV model's Test Loss: {total_loss:.4f} | Test Accuracy: {total_acc*100:.4f}%")


    test_dataset = CustomDataset(pixel_df=test.iloc[:, 1:], transform=test_transform)
    test_loader = DataLoader(test_dataset, shuffle=False, **test_loader_params)

    preds = []
    with torch.no_grad():
        for images in test_loader:
            images = images.to(device)

            sf_outputs = torch.zeros((images.size(0), CLASSES)).to(device)
            outputs = torch.zeros((images.size(0), CLASSES)).to(device)
            for model in models:
                output = model(images)

                if TTA:
                    tta_images = [tta_transform(images) for tta_transform in tta_transforms]
                    tta_outputs = [model(tta_image) for tta_image in tta_images]
                    output = [output] + tta_outputs
                    output = torch.stack(output).mean(dim=0)
                
                # soft-voting
                sf_outputs = sf_outputs + output

                # hard-voting
                _, predict_indices = torch.max(output.data, dim=1)
                pred_one_hot = torch.zeros((images.shape[0], CLASSES)).to(device)
                pred_one_hot.scatter_(1, predict_indices.unsqueeze(1), 1)
                outputs = outputs + pred_one_hot

            max_val, _ = torch.max(outputs, dim=1, keepdim=True)
            cnt = torch.sum(outputs == max_val, dim=1)
            for i in range(cnt.shape[0]):
                if cnt[i] > 1:
                    output[i] = sf_outputs[i]
            outputs = outputs / len(models)

            _, predicted = torch.max(outputs.data, 1)
            preds.extend(predicted.cpu().numpy())

    # Decode predictions
    pred_labels = encoder.inverse_transform(preds)

SV model's Test Loss: 0.0000 | Test Accuracy: 100.0000%


# Submission

In [15]:
submission = pd.read_csv('./data/sample_submission.csv')
submission['label'] = pred_labels
submission.to_csv(TRIAL_NAME + '.csv', index=False)